**A Complete Guide to BERT with Code**
----------------------------------

Introduction
------------

**BERT(Bidirectional Encoder Representations from Transformers)** 는  
Google AI Language 팀이 개발한 대규모 언어 모델(Large Language Model, LLM)로,  
자연어 처리(Natural Language Processing, NLP) 분야에 **획기적인 발전을 이끌어낸 모델**이다.  

최근 몇 년간 등장한 많은 모델들—예: **RoBERTa, ALBERT, DistilBERT** 등—은  
BERT에서 직접적으로 파생되었거나, 이를 개선한 형태이다.  

원래 BERT는 \*\*OpenAI의 GPT(Generative Pre-trained Transformer)\*\*가 발표된 직후 공개되었으며,  
두 모델 모두 전년도에 제안된 **Transformer 아키텍처**를 기반으로 구축되었다.  

* GPT는 \*\*자연어 생성(Natural Language Generation, NLG)\*\*에 중점을 두었고,  
* BERT는 \*\*자연어 이해(Natural Language Understanding, NLU)\*\*에 초점을 맞추었다.  

이 두 모델은 NLP의 지형을 완전히 바꾸었으며,
**기계학습 발전사에서 매우 중요한 이정표**로 자리매김했다.  

---

이 글에서는 BERT의 **역사적 배경**을 살펴보고, **출시 당시의 연구 환경**을 자세히 들여다본다.  
이를 통해 단순히 아키텍처 구조뿐만 아니라,  **저자들이 내린 설계 결정의 배경**과  
**BERT를 실제 산업 및 개인 프로젝트에서 어떻게 학습시키고 파인튜닝할 수 있는지**에 대한  
포괄적인 이해를 제공할 것이다.  

또한, **아키텍처를 그림으로 시각화하고**, **감정 분석(sentiment analysis) 과제에 BERT를  
파인튜닝하는 코드를 처음부터 직접 작성**해보며 실용적인 활용법도 함께 다룰 예정이다.


## 1 – BERT의 역사와 핵심 특징  
BERT 모델은 다음의 네 가지 주요 특징으로 정의할 수 있다:  
- 인코더 전용(encoder-only) 아키텍처
- 사전학습(pre-training) 방식
- 파인튜닝(fine-tuning)
- 양방향 문맥(bidirectional context)의 활용

이 각각의 특징은 BERT 논문의 저자들이 시대적 배경을 고려해 의도적으로 선택한 설계 결정이다.  
다음 섹션에서는 각 특징을 하나씩 살펴보며, 이것들이 당시의 최신 모델들(Transformer와 GPT)로부터  
어떻게 영감을 받았거나, 또는 이를 어떻게 개선하려 했는지를 설명한다.  

### 1.1 – 인코더 전용 아키텍처 (Encoder-Only Architecture)  
2017년 Transformer 모델이 처음 발표되면서,  
이를 기반으로 한 새로운 모델 개발 경쟁이 시작되었다.  

먼저 OpenAI는 2018년 6월 GPT를 발표했다.  
이는 디코더 전용(decoder-only) 모델로, **자연어 생성(NLG)**에 뛰어난 성능을 보였으며,  
이후 ChatGPT 시리즈의 기반이 되었다.  

이에 대해 Google은 그로부터 4개월 뒤,  
**자연어 이해(NLU)**에 초점을 둔 BERT를 발표하였다.  
BERT는 인코더 전용(encoder-only) 아키텍처를 사용한다.  

두 모델 모두 매우 강력한 성능을 보이지만, 수행 가능한 작업의 성격은 서로 다르다.

---

아래에 각 아키텍처의 개요를 정리하였다:  

🔹 **디코더 전용 모델 (Decoder-Only Models)**  
- **목표**:  
입력 시퀀스에 대한 응답으로 새로운 출력 시퀀스를 생성하는 것

- **개요**:  
Transformer의 디코더 블록은 입력(보통 인코더의 출력)을 바탕으로 출력 시퀀스를 생성하는 역할을 한다.  
디코더 전용 모델은 인코더 블록을 생략하고, 디코더 블록만 여러 개 쌓아 구성된다.  
이 모델들은 **프롬프트(prompt)**를 입력으로 받아  가장 가능성이 높은 다음 단어(정확히는 토큰)를
하나씩 예측하는 방식으로 출력을 생성한다.  
이 과제를 **다음 토큰 예측(Next Token Prediction, NTP)**이라고 한다.

- **적합한 과제**:
자연어 생성(NLG) 과제에 뛰어나며, 예를 들면:
대화형 챗봇, 기계 번역, 코드 생성 등

- **대중적 예시**:
GPT-3.5, GPT-4 등 ChatGPT의 기반 모델들이 이에 해당한다.

---

🔹 **인코더 전용 모델 (Encoder-Only Models)**
- **목표**:  
입력 시퀀스 내에서 단어 또는 시퀀스에 대한 예측 수행

- **개요**:  
Transformer의 인코더 블록은 입력 시퀀스를 받아  
각 단어(정확히는 토큰)에 대한 **풍부한 벡터 표현(numeric vector representation)**을 생성한다.  
인코더 전용 모델은 디코더를 제거하고, 인코더 블록만 여러 개 쌓아 구성된다.  
이 모델은 GPT처럼 프롬프트에 응답하는 방식이 아니라, 주어진 입력 시퀀스 자체에 대한 예측  
(예: 마스킹된 단어 예측)을 수행한다.  
디코더가 없기 때문에, 새로운 텍스트 생성에는 적합하지 않다.  

- **적합한 과제:**
자연어 이해(NLU) 과제에 강하며, 예를 들면: 개체명 인식(NER), 감정 분석(sentiment analysis) 등  

- **모델 특성:**  
인코더 블록이 만들어내는 깊이 있는 벡터 표현이 BERT의 핵심 이해력의 원천이다.  

- **논문 인용:**  
BERT 저자들은 다음과 같이 주장했다:
> “디코더 전용 아키텍처는 문장 수준 과제에 최적이 아니며,  
질문 응답과 같은 토큰 수준 과제에 파인튜닝을 적용할 때는 매우 불리할 수 있다.” [1]  
> **참고:** 
기술적으로는 BERT로도 텍스트를 생성할 수 있지만,  
애초에 생성 작업을 위한 구조가 아니므로 GPT 같은 디코더 전용 모델과는 성능 면에서 비교가 되지 않는다.


**Architecture Diagrams for the Transformer, GPT, and BERT:**
---------------------------------------------------------

아래에는 지금까지 살펴본 **세 가지 모델(Transformer, GPT, BERT)**의 아키텍처 도식이 나와 있다.  
이 그림은 원래의 Transformer 논문 "Attention is All You Need" [2]에 나온 구조를 BERT와 GPT에   
맞게 재구성한 것이다. 원래 Transformer는 인코더와 디코더가 각각 **6개의 블록(N = 6)** 으로 구성되어 있다.  
GPT는 디코더 블록만 쌓은 구조, BERT는 인코더 블록만 쌓은 구조이다.

![fig_1](../images/llm_bert1.webp)

A comparison of the architectures for the Transformer, GPT, and BERT. Image adapted by author from the Transformer architecture diagram in the "Attention is All You Need" paper [2].

### 1.2 - Pre-trainning Approach  
GPT는 여러 측면에서 BERT의 개발에 영향을 주었다.  
GPT는 최초의 디코더 전용(Decoder-only) 트랜스포머 파생 모델일 뿐만 아니라,  
모델 사전학습(pre-training) 개념을 대중화시킨 모델이기도 하다.  

사전학습이란, 하나의 대규모 모델을 훈련시켜 언어에 대한 광범위한 이해 능력  
(예: 단어 사용, 문법 패턴 등)을 습득하게 하고, 이를 다양한 과제에 사용할 수 있는  
과제 비특화(task-agnostic) 기반 모델로 만드는 것을 말한다.  

도식에서 보면, 이 기반 모델은 선형 계층(linear layer) 아래에 위치한 구성요소들로 이루어진다  
(보라색으로 표시). 사전학습이 완료되면, 이 모델의 복사본을 가져와 특정 과제에 맞게  
파인튜닝(fine-tuning) 할 수 있다.  

파인튜닝은 보통 선형 계층—즉, 분류 헤드(classification head) 또는   
간단한 피드포워드 신경망만을 학습하는 방식이며, 그 외 기반 모델의 가중치와 바이어스는  
**변경하지 않고 고정(frozen)** 된다.

비유(Analogy):
--------------
예를 들어 감정 분석 과제를 생각해보자.  
여기서 목표는 입력된 텍스트가 긍정적인지 부정적인지 분류하는 것이다.  

예시:  
“I loved this movie” → 긍정  
“I hated this movie” → 부정  

전통적인 언어 모델링 접근법에서는, 이러한 과제 하나를 위해 처음부터 새로운 모델 아키텍처를  
설계하고 훈련해야 했다. 이것은 마치, 누군가에게 영어를 처음부터 가르친 뒤, 수많은 영화 리뷰를  
보여주며 감정 분류를 익히게 하는 것과 같다.  
→ 느리고, 비용이 많이 들며, 많은 학습 데이터가 필요하다. 게다가, 그렇게 만들어진 모델은   
해당 작업 하나에만 특화되어 있다.

반면, 사전학습 접근 방식에서는 이미 영어를 유창하게 이해하고 있는 사람에게 단 몇 개의 리뷰만  
보여주며 과제를 학습시키는 방식이다.  
→ 훨씬 빠르고 효율적이며, 다양한 과제에 재활용이 가능하다.  

초기 사전학습 시도
-------------------

사전학습 개념은 OpenAI가 처음 개발한 것은 아니며, 그 이전에도 여러 연구자들에 의해 시도된 바 있다.  
대표적인 예로는 ELMo (Embeddings from Language Models) 모델이 있으며, 이는 Allen Institute 연구자들이  
개발하였다 [3].  

하지만 이러한 초기 시도들에도 불구하고, OpenAI만큼 사전학습의 효과를 명확히 입증한 사례는 없었다.  
OpenAI의 대표 논문에서는 다음과 같이 언급된다:  

> “우리의 과제-비특화 모델은, 각 과제에 특화된 아키텍처로 학습된 모델보다 더 뛰어난 성능을 보였으며,  
기존 최고 성능(state of the art)을 크게 능가했다.” [4]

이러한 발견은 사전학습 패러다임을 언어 모델링 분야의 주류 접근 방식으로 자리매김하게 만들었다.  
BERT의 저자들 또한 이 흐름에 발맞춰, 사전학습 기반 접근법을 완전히 채택하였다.

### 1.3 – Model Fine-tuning

Benefits of Fine-tuning:
------------------------

오늘날 파인튜닝은 매우 일반적인 방식이 되었지만, 이 접근법이 두각을 나타내기 시작한 시점이  
그리 오래되지 않았다는 사실은 종종 간과되곤 한다. 2018년 이전까지는 각기 다른 자연어 처리(NLP)  
과제를 위해 새로운 모델 아키텍처를 설계하는 것이 일반적이었다.  
사전학습(pre-training)으로의 전환은 모델 개발에 필요한 학습 시간과 계산 비용을 획기적으로 줄였을 뿐 아니라,    
필요한 학습 데이터의 양도 크게 감소시켰다. 기존에는 언어 모델을 처음부터 새롭게 설계하고 학습시켜야 했다면,   
이제는 GPT와 같은 범용 모델을 소량의 과제 특화 데이터(task-specific data)로   
짧은 시간 내에 파인튜닝하는 것이 가능해졌다.

과제에 따라 분류기 헤드(classification head)의 출력 뉴런 수를 조정할 수 있다.  
이는 감정 분석과 같은 분류 과제에 유용하다. 예를 들어, BERT 모델의 출력이 리뷰가   
긍정적인지 부정적인지를 예측하는 것이라면, 출력층은 두 개의 뉴런을 가지도록 설정할 수 있다.  
각 뉴런의 활성화 값은 해당 리뷰가 긍정 또는 부정일 확률을 나타낸다.  
만약 10개의 클래스를 가진 다중 클래스 분류 과제라면, 출력층을 10개의 뉴런으로 구성하면 된다.  
이처럼 BERT는 다양한 다운스트림 과제에 활용 가능한 유연성을 갖추게 된다.


Fine-tuning in BERT:
--------------------

BERT는 GPT의 전례를 따르며, 사전학습과 파인튜닝이라는 접근 방식을 채택하였다.   
구글은 하드웨어 제약을 고려해 모델 크기 선택의 유연성을 제공하기 위해 BERT의   
두 가지 버전을 공개하였다: Base와 Large. 두 버전 모두 수많은 TPU(텐서 처리 유닛)를  
활용하여 약 4일간 사전학습을 거쳤으며, BERT Base는 16개의 TPU, BERT Large는 64개의 TPU에서 학습되었다.   
대부분의 연구자, 취미 개발자, 산업 현업자에게는 이러한 규모의 학습은 현실적으로 어렵다.   
따라서, 특정 과제에 대해 몇 시간만 투자하여 범용 모델을 파인튜닝하는 방식이 훨씬 매력적인 대안으로 여겨진다.  

원래의 BERT 아키텍처는 다양한 과제와 데이터셋에 대해 수천 번의 파인튜닝이 이루어졌으며,   
이 중 다수는 Hugging Face와 같은 플랫폼에서 공개적으로 다운로드할 수 있다 [5].

### 1.4 – Use of Bidirectional Context  

언어 모델로서 BERT는 이전 단어들이 주어졌을 때 특정 단어가 등장할 확률을 예측한다.  
이 기본적인 특징은 모델의 아키텍처나 목표 과제와 무관하게 모든 언어 모델에 공통된 속성이다.  
그러나 이러한 확률을 어떻게 활용하느냐에 따라 모델의 과제 특화적 행동이 결정된다.   
예를 들어, GPT는 주어진 단어 시퀀스에서 다음에 올 가능성이 가장 높은 단어를 예측하도록 학습된다.  
즉, 이전 단어들이 주어졌을 때 다음 단어를 예측하는 것이다.     

다른 모델들은 감정 분석처럼 입력 시퀀스의 감정을 "긍정" 혹은 "부정"과 같은 텍스트 레이블로  
분류하도록 학습될 수도 있다. 텍스트에 대해 의미 있는 예측을 하기 위해서는,   
특히 자연어 이해(NLU) 과제에서는 주변 문맥의 이해가 필수적이다.   
BERT는 이와 같은 이해를 가능하게 해주는 핵심 속성 중 하나인   
**양방향성(bidirectionality)** 을 통해 이를 보장한다.

양방향성은 아마도 BERT의 가장 중요한 특징으로, 자연어 이해 과제에서의 높은 성능을 이끄는 핵심 요소이자,  
인코더 전용 아키텍처를 채택한 주된 이유이기도 하다. 트랜스포머 인코더의 셀프 어텐션 메커니즘은  
양방향 문맥을 계산할 수 있지만, 디코더는 단방향 문맥만을 생성한다는 점에서 동일하지 않다.   
BERT의 저자들은 GPT에는 이러한 양방향성이 없기 때문에, 언어 표현의 깊이에 있어서 BERT만큼 도달하지 못한다고 주장했다.    

양방향성의 정의:
-----------------
그렇다면 "양방향" 문맥이란 정확히 무엇을 의미할까? 여기서 양방향이란, 입력 시퀀스의   
각 단어가 **이전 단어들(왼쪽 문맥)** 과 이후 단어들(오른쪽 문맥) 모두로부터 문맥 정보를 얻을 수 있다는 뜻이다.  
기술적인 용어로 말하면, 어텐션 메커니즘이 각 단어에 대해 **앞뒤 토큰 모두에 주의를 기울일 수 있다(attend)** 는
의미다. 좀 더 구체적으로 설명하면, BERT는 GPT처럼 새로운 시퀀스를 생성하지 않고,   
주어진 입력 시퀀스 안의 단어를 예측하는 데 초점을 둔다. 따라서 BERT가 어떤 단어를 예측할 때는  
주변 모든 단어로부터 문맥적인 단서를 활용할 수 있다. 이런 방식으로 좌우 양방향의 문맥 정보를  
활용하여 보다 정교하고 신뢰성 있는 예측을 가능하게 한다.

이에 반해 GPT와 같은 디코더 전용 모델은, 새로운 단어들을 하나씩 예측하며 출력 시퀀스를 생성하는  
것이 목적이다. 이 과정에서 각 단어는 **이전 단어들(왼쪽 문맥)**만을 참고할 수 있으며, 이후 단어들(오른쪽 문맥)은  
아직 생성되지 않았기 때문에 활용할 수 없다. 이런 이유로 이러한 모델들을 **단방향(unidirectional)** 이라고 부른다.






나의 말:
